In [45]:
from matplotlib import pyplot as plt
%matplotlib inline
# 設定圖形大小; DPI越大圖越大
plt.rcParams["figure.dpi"] = 150
import seaborn as sns

import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as smm
import statsmodels.stats.outliers_influence as sso

In [46]:
data = pd.read_csv('data/tracks_no_duplicate.csv', encoding='utf-8', index_col=False)

In [47]:
one_hot = pd.get_dummies(data['genre'], prefix='genre').iloc[:, :-1]

In [48]:
df = pd.concat([data.iloc[:, 4:-1], one_hot], axis=1)

In [62]:
formula = 'popularity ~ '
for i, c in enumerate(df.columns[1:]):
    if c == 'release_date':
        continue
    if i != 0:
        formula += ' + '
    formula += f'Q("{c}")'

In [63]:
model = smf.ols(formula, data=df).fit()

In [64]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             popularity   R-squared:                       0.677
Model:                            OLS   Adj. R-squared:                  0.674
Method:                 Least Squares   F-statistic:                     182.0
Date:                Sat, 23 May 2020   Prob (F-statistic):               0.00
Time:                        20:21:27   Log-Likelihood:                -41075.
No. Observations:               10963   AIC:                         8.240e+04
Df Residuals:                   10837   BIC:                         8.332e+04
Df Model:                         125                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       43.5138      1.537     28.309      0.000      40.501      46.527
Q("acousticness")               -1.0151      0.528     -1.923      0.055      -2.050       0.020
Q("danceability")                2.2114      0.848      2.608      0.009       0.549       3.873
Q("energy")                     -3.3839      0.884     -3.828      0.000      -5.116      -1.651
Q("instrumentalness")           -2.1547      0.436     -4.944      0.000      -3.009      -1.300
Q("liveness")                   -2.1154      0.614     -3.446      0.001      -3.319      -0.912
Q("speechiness")                -4.2002      1.328     -3.162      0.002      -6.804      -1.597
Q("duration_ms")              9.338e-08   1.08e-06      0.087      0.931   -2.02e-06    2.21e-06
Q("key")                        -0.0186      0.028     -0.660      0.509      -0.074       0.037
Q("loadness")                    0.2280      0.039      5.878      0.000       0.152       0.304
Q("mode")                       -0.6665      0.215     -3.093      0.002      -1.089      -0.244
Q("tempo")                   -8.679e-05      0.004     -0.024      0.981      -0.007       0.007
Q("valence")                    -0.5078      0.526     -0.966      0.334      -1.538       0.522
Q("available_market")            0.0691      0.005     14.524      0.000       0.060       0.078
Q("explicit")                    1.6244      0.414      3.921      0.000       0.812       2.436
Q("genre_acoustic")              0.5366      1.472      0.364      0.716      -2.350       3.423
Q("genre_afrobeat")            -10.9914      1.485     -7.400      0.000     -13.903      -8.080
Q("genre_alt-rock")             17.1165      1.487     11.512      0.000      14.202      20.031
Q("genre_alternative")          19.6292      1.513     12.970      0.000      16.663      22.596
Q("genre_ambient")               4.9480      1.525      3.245      0.001       1.959       7.937
Q("genre_anime")                -1.4090      1.475     -0.955      0.339      -4.300       1.482
Q("genre_black-metal")         -12.3711      1.493     -8.285      0.000     -15.298      -9.444
Q("genre_bluegrass")            -8.9723      1.478     -6.070      0.000     -11.870      -6.075
Q("genre_blues")                 7.1959      1.497      4.807      0.000       4.262      10.130
Q("genre_brazil")               -6.6913      1.490     -4.491      0.000      -9.612      -3.771
Q("genre_breakbeat")           -17.8019      1.491    -11.938      0.000     -20.725     -14.879
Q("genre_british")               6.3538      1.486      4.277      0.000       3.441       9.266
Q("genre_cantopop")             -8.3031      1.471     -5.644      0.000     -11.187      -5.420
Q("genre_chicago-house")       -20.7589      1.503    -13.807      0.000    